# Project 17: Multilanguage NLP Pipeline

**Build a system that supports multiple languages in text processing**

In this comprehensive tutorial, we'll build a complete multilingual NLP pipeline that can:
- Detect languages automatically
- Tokenize text in multiple languages
- Clean and preprocess multilingual text
- Perform POS tagging across languages
- Remove language-specific stopwords
- Lemmatize words in different languages
- Generate multilingual embeddings
- Perform cross-lingual text classification

**Pipeline Overview:**
```
Raw Text → Language Detection → Tokenization → Text Cleaning → POS Tagging → 
Stopword Removal → Lemmatization → Cleaned Text → Embeddings → ML Model
```

**Languages we'll support:** English, Spanish, French, German, Italian, Portuguese

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Language Detection](#2-language-detection)
3. [Multilingual Tokenization](#3-multilingual-tokenization)
4. [Text Cleaning](#4-text-cleaning)
5. [POS Tagging Across Languages](#5-pos-tagging-across-languages)
6. [Stopword Removal](#6-stopword-removal)
7. [Lemmatization](#7-lemmatization)
8. [Complete Pipeline Class](#8-complete-pipeline-class)
9. [Multilingual Embeddings](#9-multilingual-embeddings)
10. [Cross-Lingual Text Classification](#10-cross-lingual-text-classification)
11. [Translation System](#11-translation-system)
12. [Summary](#12-summary)

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q langdetect spacy transformers sentencepiece torch nltk
!pip install -q sacremoses  # For translation tokenization

# Download spaCy models for multiple languages
!python -m spacy download en_core_web_sm  # English
!python -m spacy download es_core_news_sm  # Spanish
!python -m spacy download fr_core_news_sm  # French
!python -m spacy download de_core_news_sm  # German
!python -m spacy download it_core_news_sm  # Italian
!python -m spacy download pt_core_news_sm  # Portuguese

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import spacy
from langdetect import detect, detect_langs
import nltk
from nltk.corpus import stopwords

# Download NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# Deep learning
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel,
    MarianMTModel, 
    MarianTokenizer,
    pipeline
)

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
# Load spaCy models for different languages
# We'll use a dictionary to manage them

SPACY_MODELS = {
    'en': 'en_core_web_sm',
    'es': 'es_core_news_sm',
    'fr': 'fr_core_news_sm',
    'de': 'de_core_news_sm',
    'it': 'it_core_news_sm',
    'pt': 'pt_core_news_sm'
}

LANGUAGE_NAMES = {
    'en': 'English',
    'es': 'Spanish',
    'fr': 'French',
    'de': 'German',
    'it': 'Italian',
    'pt': 'Portuguese'
}

def load_spacy_model(lang_code):
    """Load spaCy model for a given language code."""
    if lang_code in SPACY_MODELS:
        try:
            return spacy.load(SPACY_MODELS[lang_code])
        except OSError:
            print(f"Model for {lang_code} not found. Using English as fallback.")
            return spacy.load('en_core_web_sm')
    else:
        print(f"Language {lang_code} not supported. Using English.")
        return spacy.load('en_core_web_sm')

# Load English model as default
nlp_en = spacy.load('en_core_web_sm')
print("Default English model loaded!")

## 2. Language Detection

The first step in a multilingual pipeline is identifying what language the text is in. This determines which models and resources to use.

**Methods:**
- **langdetect**: Fast, statistical approach based on character n-grams
- **Transformer-based**: More accurate but slower

In [ ]:
class LanguageDetector:
    """
    Detect language of text using multiple methods.
    
    Methods:
    - langdetect: Fast, statistical (default)
    - transformer: More accurate using XLM-RoBERTa
    """
    
    def __init__(self, method='langdetect'):
        self.method = method
        self.supported_languages = list(LANGUAGE_NAMES.keys())
        
        if method == 'transformer':
            # Load language detection pipeline
            self.detector = pipeline(
                'text-classification',
                model='papluca/xlm-roberta-base-language-detection',
                device=0 if torch.cuda.is_available() else -1
            )
    
    def detect(self, text):
        """
        Detect the language of given text.
        
        Returns:
            tuple: (language_code, confidence)
        """
        if not text or len(text.strip()) < 3:
            return ('unknown', 0.0)
        
        if self.method == 'langdetect':
            return self._detect_langdetect(text)
        else:
            return self._detect_transformer(text)
    
    def _detect_langdetect(self, text):
        """Use langdetect library for detection."""
        try:
            langs = detect_langs(text)
            top_lang = langs[0]
            return (top_lang.lang, top_lang.prob)
        except:
            return ('unknown', 0.0)
    
    def _detect_transformer(self, text):
        """Use transformer model for detection."""
        result = self.detector(text[:512])[0]  # Limit text length
        return (result['label'], result['score'])
    
    def detect_batch(self, texts):
        """Detect languages for multiple texts."""
        return [self.detect(text) for text in texts]

# Initialize detector
lang_detector = LanguageDetector(method='langdetect')

# Test with sample texts in different languages
sample_texts = [
    "Hello, how are you today? This is a test in English.",
    "Hola, ¿cómo estás hoy? Esta es una prueba en español.",
    "Bonjour, comment allez-vous aujourd'hui? Ceci est un test en français.",
    "Hallo, wie geht es Ihnen heute? Dies ist ein Test auf Deutsch.",
    "Ciao, come stai oggi? Questo è un test in italiano.",
    "Olá, como você está hoje? Este é um teste em português."
]

print("Language Detection Results:")
print("=" * 60)
for text in sample_texts:
    lang, conf = lang_detector.detect(text)
    lang_name = LANGUAGE_NAMES.get(lang, lang)
    print(f"Language: {lang_name:12} | Confidence: {conf:.2%}")
    print(f"  Text: {text[:50]}...")
    print()

## 3. Multilingual Tokenization

Tokenization varies by language:
- **English**: Space-based with punctuation handling
- **German**: Compound words need special handling
- **Chinese/Japanese**: No spaces between words
- **Romance languages**: Similar to English but with contractions

We'll use spaCy for tokenization as it has language-specific rules.

In [ ]:
class MultilingualTokenizer:
    """
    Tokenize text in multiple languages using spaCy.
    
    Features:
    - Language-specific tokenization rules
    - Handles punctuation, contractions, compounds
    - Returns token objects with metadata
    """
    
    def __init__(self):
        self.nlp_models = {}
        self.lang_detector = LanguageDetector()
    
    def _get_nlp(self, lang_code):
        """Get or load spaCy model for language."""
        if lang_code not in self.nlp_models:
            self.nlp_models[lang_code] = load_spacy_model(lang_code)
        return self.nlp_models[lang_code]
    
    def tokenize(self, text, lang=None):
        """
        Tokenize text into words.
        
        Args:
            text: Input text
            lang: Language code (auto-detected if None)
            
        Returns:
            list: List of token strings
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        return [token.text for token in doc]
    
    def tokenize_detailed(self, text, lang=None):
        """
        Tokenize with detailed information.
        
        Returns:
            list: List of dicts with token details
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        
        return [{
            'text': token.text,
            'lemma': token.lemma_,
            'pos': token.pos_,
            'is_punct': token.is_punct,
            'is_stop': token.is_stop,
            'is_alpha': token.is_alpha
        } for token in doc]

# Initialize tokenizer
tokenizer = MultilingualTokenizer()

# Test tokenization
test_text_en = "I can't believe it's already 2024! Machine learning is amazing."
test_text_de = "Ich kann es nicht glauben! Maschinelles Lernen ist erstaunlich."

print("English Tokenization:")
tokens_en = tokenizer.tokenize(test_text_en, lang='en')
print(f"Text: {test_text_en}")
print(f"Tokens: {tokens_en}")
print()

print("German Tokenization:")
tokens_de = tokenizer.tokenize(test_text_de, lang='de')
print(f"Text: {test_text_de}")
print(f"Tokens: {tokens_de}")

In [ ]:
# Detailed tokenization example
print("Detailed Token Analysis (English):")
print("=" * 70)

details = tokenizer.tokenize_detailed(test_text_en, lang='en')
df_tokens = pd.DataFrame(details)
print(df_tokens.to_string())

## 4. Text Cleaning

Text cleaning removes noise while preserving meaningful content. Multilingual considerations:
- **Accented characters**: Keep them! (café, résumé, naïve)
- **Unicode normalization**: Standardize character representations
- **Language-specific patterns**: Email, URLs, mentions are universal

In [ ]:
import unicodedata

class MultilingualTextCleaner:
    """
    Clean text while preserving language-specific characters.
    
    Features:
    - Remove URLs, emails, mentions
    - Normalize unicode
    - Handle emojis
    - Preserve accented characters
    """
    
    def __init__(self):
        # Regex patterns
        self.url_pattern = re.compile(r'https?://\S+|www\.\S+')
        self.email_pattern = re.compile(r'\S+@\S+\.\S+')
        self.mention_pattern = re.compile(r'@\w+')
        self.hashtag_pattern = re.compile(r'#\w+')
        self.number_pattern = re.compile(r'\d+')
        self.extra_space_pattern = re.compile(r'\s+')
        
        # Emoji pattern (covers most common emojis)
        self.emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]+",
            flags=re.UNICODE
        )
    
    def clean(self, text, 
              remove_urls=True,
              remove_emails=True,
              remove_mentions=True,
              remove_hashtags=False,
              remove_numbers=False,
              remove_emojis=True,
              lowercase=True,
              normalize_unicode=True):
        """
        Clean text with configurable options.
        
        Args:
            text: Input text
            remove_*: Boolean flags for what to remove
            lowercase: Convert to lowercase
            normalize_unicode: Apply NFC normalization
            
        Returns:
            str: Cleaned text
        """
        if not text:
            return ""
        
        # Unicode normalization (NFC keeps accented chars as single chars)
        if normalize_unicode:
            text = unicodedata.normalize('NFC', text)
        
        # Remove patterns
        if remove_urls:
            text = self.url_pattern.sub(' ', text)
        if remove_emails:
            text = self.email_pattern.sub(' ', text)
        if remove_mentions:
            text = self.mention_pattern.sub(' ', text)
        if remove_hashtags:
            text = self.hashtag_pattern.sub(' ', text)
        if remove_numbers:
            text = self.number_pattern.sub(' ', text)
        if remove_emojis:
            text = self.emoji_pattern.sub(' ', text)
        
        # Lowercase
        if lowercase:
            text = text.lower()
        
        # Remove extra whitespace
        text = self.extra_space_pattern.sub(' ', text)
        text = text.strip()
        
        return text
    
    def clean_batch(self, texts, **kwargs):
        """Clean multiple texts."""
        return [self.clean(text, **kwargs) for text in texts]

# Initialize cleaner
cleaner = MultilingualTextCleaner()

# Test with noisy multilingual text
noisy_texts = [
    "Check out https://example.com! Contact me@email.com #NLP @user 😀",
    "¡Hola! Visita https://sitio.es ¿Qué tal? 🎉 #español",
    "Café, résumé, naïve - keeping accents! Visit http://test.fr",
    "Größe und Äpfel - German umlauts! 📧 test@mail.de"
]

print("Text Cleaning Results:")
print("=" * 70)
for text in noisy_texts:
    cleaned = cleaner.clean(text)
    print(f"Original: {text}")
    print(f"Cleaned:  {cleaned}")
    print()

## 5. POS Tagging Across Languages

Part-of-Speech (POS) tagging identifies grammatical categories:
- **NOUN**: Nouns
- **VERB**: Verbs
- **ADJ**: Adjectives
- **ADV**: Adverbs
- **PROPN**: Proper nouns

spaCy uses Universal Dependencies tagset - consistent across languages!

In [ ]:
class MultilingualPOSTagger:
    """
    POS tagging for multiple languages using spaCy.
    
    Uses Universal Dependencies tagset for consistency.
    """
    
    # Universal POS tags description
    POS_DESCRIPTIONS = {
        'ADJ': 'Adjective',
        'ADP': 'Adposition (preposition)',
        'ADV': 'Adverb',
        'AUX': 'Auxiliary verb',
        'CCONJ': 'Coordinating conjunction',
        'DET': 'Determiner',
        'INTJ': 'Interjection',
        'NOUN': 'Noun',
        'NUM': 'Numeral',
        'PART': 'Particle',
        'PRON': 'Pronoun',
        'PROPN': 'Proper noun',
        'PUNCT': 'Punctuation',
        'SCONJ': 'Subordinating conjunction',
        'SYM': 'Symbol',
        'VERB': 'Verb',
        'X': 'Other'
    }
    
    def __init__(self):
        self.nlp_models = {}
        self.lang_detector = LanguageDetector()
    
    def _get_nlp(self, lang_code):
        """Get or load spaCy model for language."""
        if lang_code not in self.nlp_models:
            self.nlp_models[lang_code] = load_spacy_model(lang_code)
        return self.nlp_models[lang_code]
    
    def tag(self, text, lang=None):
        """
        Tag text with POS labels.
        
        Args:
            text: Input text
            lang: Language code (auto-detected if None)
            
        Returns:
            list: List of (token, pos_tag) tuples
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        
        return [(token.text, token.pos_) for token in doc]
    
    def tag_detailed(self, text, lang=None):
        """
        Tag with detailed morphological information.
        
        Returns:
            list: List of dicts with detailed POS info
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        
        return [{
            'token': token.text,
            'pos': token.pos_,
            'pos_desc': self.POS_DESCRIPTIONS.get(token.pos_, 'Unknown'),
            'tag': token.tag_,  # Fine-grained tag
            'dep': token.dep_,  # Dependency relation
            'morph': str(token.morph)  # Morphological features
        } for token in doc]
    
    def get_pos_distribution(self, text, lang=None):
        """Get distribution of POS tags in text."""
        tags = self.tag(text, lang)
        pos_counts = Counter(tag for _, tag in tags)
        return dict(pos_counts)

# Initialize POS tagger
pos_tagger = MultilingualPOSTagger()

# Test on multiple languages
test_sentences = {
    'en': "The quick brown fox jumps over the lazy dog.",
    'es': "El rápido zorro marrón salta sobre el perro perezoso.",
    'fr': "Le renard brun rapide saute par-dessus le chien paresseux.",
    'de': "Der schnelle braune Fuchs springt über den faulen Hund."
}

print("POS Tagging Across Languages:")
print("=" * 70)

for lang, sentence in test_sentences.items():
    print(f"\n{LANGUAGE_NAMES[lang]}:")
    print(f"Sentence: {sentence}")
    tags = pos_tagger.tag(sentence, lang)
    print("Tags:", [(t, p) for t, p in tags if p != 'PUNCT'])

In [ ]:
# Visualize POS distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, (lang, sentence) in zip(axes.flatten(), test_sentences.items()):
    dist = pos_tagger.get_pos_distribution(sentence, lang)
    # Remove punctuation from visualization
    dist.pop('PUNCT', None)
    dist.pop('SPACE', None)
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(dist)))
    ax.bar(dist.keys(), dist.values(), color=colors)
    ax.set_title(f'{LANGUAGE_NAMES[lang]} POS Distribution')
    ax.set_xlabel('POS Tag')
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Stopword Removal

Stopwords are common words that carry little semantic meaning:
- **English**: the, is, at, which, on
- **Spanish**: el, la, de, en, y
- **French**: le, la, de, et, est

We'll combine stopwords from NLTK and spaCy for comprehensive coverage.

In [ ]:
class MultilingualStopwordRemover:
    """
    Remove stopwords from text in multiple languages.
    
    Combines stopwords from NLTK and spaCy.
    """
    
    # NLTK language codes mapping
    NLTK_LANG_MAP = {
        'en': 'english',
        'es': 'spanish',
        'fr': 'french',
        'de': 'german',
        'it': 'italian',
        'pt': 'portuguese'
    }
    
    def __init__(self):
        self.stopwords_cache = {}
        self.nlp_models = {}
        self.lang_detector = LanguageDetector()
    
    def _get_stopwords(self, lang_code):
        """Get combined stopwords for language."""
        if lang_code in self.stopwords_cache:
            return self.stopwords_cache[lang_code]
        
        combined_stopwords = set()
        
        # NLTK stopwords
        if lang_code in self.NLTK_LANG_MAP:
            try:
                nltk_stops = set(stopwords.words(self.NLTK_LANG_MAP[lang_code]))
                combined_stopwords.update(nltk_stops)
            except:
                pass
        
        # spaCy stopwords
        if lang_code not in self.nlp_models:
            self.nlp_models[lang_code] = load_spacy_model(lang_code)
        
        nlp = self.nlp_models[lang_code]
        spacy_stops = nlp.Defaults.stop_words
        combined_stopwords.update(spacy_stops)
        
        self.stopwords_cache[lang_code] = combined_stopwords
        return combined_stopwords
    
    def remove_stopwords(self, text, lang=None, return_removed=False):
        """
        Remove stopwords from text.
        
        Args:
            text: Input text
            lang: Language code (auto-detected if None)
            return_removed: If True, also return removed words
            
        Returns:
            str or tuple: Cleaned text (and removed words if requested)
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        stops = self._get_stopwords(lang)
        
        # Tokenize and filter
        if lang not in self.nlp_models:
            self.nlp_models[lang] = load_spacy_model(lang)
        
        nlp = self.nlp_models[lang]
        doc = nlp(text)
        
        kept_tokens = []
        removed_tokens = []
        
        for token in doc:
            if token.text.lower() in stops or token.is_stop:
                removed_tokens.append(token.text)
            else:
                kept_tokens.append(token.text)
        
        result = ' '.join(kept_tokens)
        
        if return_removed:
            return result, removed_tokens
        return result
    
    def get_stopword_count(self, lang_code):
        """Get number of stopwords for a language."""
        return len(self._get_stopwords(lang_code))

# Initialize stopword remover
stopword_remover = MultilingualStopwordRemover()

# Show stopword counts per language
print("Stopword Counts by Language:")
print("=" * 40)
for lang in LANGUAGE_NAMES.keys():
    count = stopword_remover.get_stopword_count(lang)
    print(f"{LANGUAGE_NAMES[lang]:12}: {count:4} stopwords")

In [ ]:
# Test stopword removal
test_texts = {
    'en': "The quick brown fox jumps over the lazy dog in the park.",
    'es': "El rápido zorro marrón salta sobre el perro perezoso en el parque.",
    'fr': "Le renard brun rapide saute par-dessus le chien paresseux dans le parc.",
    'de': "Der schnelle braune Fuchs springt über den faulen Hund im Park."
}

print("Stopword Removal Results:")
print("=" * 70)

for lang, text in test_texts.items():
    cleaned, removed = stopword_remover.remove_stopwords(text, lang, return_removed=True)
    print(f"\n{LANGUAGE_NAMES[lang]}:")
    print(f"Original: {text}")
    print(f"Cleaned:  {cleaned}")
    print(f"Removed:  {removed}")

## 7. Lemmatization

Lemmatization reduces words to their base/dictionary form:
- **running → run**
- **better → good**
- **corriendo → correr** (Spanish)

Unlike stemming, lemmatization uses vocabulary and morphological analysis for accurate results.

In [ ]:
class MultilingualLemmatizer:
    """
    Lemmatize text in multiple languages using spaCy.
    
    Lemmatization reduces words to their dictionary form
    while considering context and part of speech.
    """
    
    def __init__(self):
        self.nlp_models = {}
        self.lang_detector = LanguageDetector()
    
    def _get_nlp(self, lang_code):
        """Get or load spaCy model for language."""
        if lang_code not in self.nlp_models:
            self.nlp_models[lang_code] = load_spacy_model(lang_code)
        return self.nlp_models[lang_code]
    
    def lemmatize(self, text, lang=None):
        """
        Lemmatize text.
        
        Args:
            text: Input text
            lang: Language code (auto-detected if None)
            
        Returns:
            str: Lemmatized text
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        
        return ' '.join([token.lemma_ for token in doc])
    
    def lemmatize_detailed(self, text, lang=None):
        """
        Lemmatize with word-by-word comparison.
        
        Returns:
            list: List of (original, lemma) tuples
        """
        if lang is None:
            lang, _ = self.lang_detector.detect(text)
        
        nlp = self._get_nlp(lang)
        doc = nlp(text)
        
        return [(token.text, token.lemma_) for token in doc]
    
    def get_lemma_mapping(self, text, lang=None):
        """
        Get mapping of changed words only.
        
        Returns:
            dict: {original: lemma} for changed words
        """
        pairs = self.lemmatize_detailed(text, lang)
        return {orig: lemma for orig, lemma in pairs 
                if orig.lower() != lemma.lower()}

# Initialize lemmatizer
lemmatizer = MultilingualLemmatizer()

# Test lemmatization across languages
test_sentences = {
    'en': "The cats were running and jumping over the fallen leaves.",
    'es': "Los gatos estaban corriendo y saltando sobre las hojas caídas.",
    'fr': "Les chats couraient et sautaient par-dessus les feuilles tombées.",
    'de': "Die Katzen rannten und sprangen über die gefallenen Blätter."
}

print("Lemmatization Results:")
print("=" * 70)

for lang, sentence in test_sentences.items():
    lemmatized = lemmatizer.lemmatize(sentence, lang)
    changes = lemmatizer.get_lemma_mapping(sentence, lang)
    
    print(f"\n{LANGUAGE_NAMES[lang]}:")
    print(f"Original:   {sentence}")
    print(f"Lemmatized: {lemmatized}")
    print(f"Changes:    {changes}")

## 8. Complete Pipeline Class

Now let's combine all components into a unified multilingual NLP pipeline that follows the flow:

```
Raw Text → Language Detection → Tokenization → Text Cleaning → 
POS Tagging → Stopword Removal → Lemmatization → Cleaned Text → ML Model
```

In [ ]:
class MultilingualNLPPipeline:
    """
    Complete multilingual NLP preprocessing pipeline.
    
    Pipeline stages:
    1. Language Detection
    2. Tokenization
    3. Text Cleaning
    4. POS Tagging
    5. Stopword Removal
    6. Lemmatization
    
    Output ready for embeddings or ML models.
    """
    
    def __init__(self):
        self.lang_detector = LanguageDetector()
        self.tokenizer = MultilingualTokenizer()
        self.cleaner = MultilingualTextCleaner()
        self.pos_tagger = MultilingualPOSTagger()
        self.stopword_remover = MultilingualStopwordRemover()
        self.lemmatizer = MultilingualLemmatizer()
        
        self.supported_languages = list(LANGUAGE_NAMES.keys())
    
    def process(self, text, 
                lang=None,
                clean=True,
                remove_stopwords=True,
                lemmatize=True,
                lowercase=True,
                remove_punctuation=True,
                return_details=False):
        """
        Process text through the complete pipeline.
        
        Args:
            text: Input text
            lang: Language code (auto-detected if None)
            clean: Apply text cleaning
            remove_stopwords: Remove stopwords
            lemmatize: Apply lemmatization
            lowercase: Convert to lowercase
            remove_punctuation: Remove punctuation tokens
            return_details: Return detailed processing info
            
        Returns:
            str or dict: Processed text (or detailed results)
        """
        details = {
            'original': text,
            'steps': []
        }
        
        # Step 1: Language Detection
        if lang is None:
            lang, conf = self.lang_detector.detect(text)
            details['detected_language'] = lang
            details['language_confidence'] = conf
        else:
            details['detected_language'] = lang
            details['language_confidence'] = 1.0
        
        details['steps'].append(('language_detection', lang))
        
        # Step 2: Text Cleaning
        if clean:
            text = self.cleaner.clean(text, lowercase=lowercase)
            details['steps'].append(('cleaning', text))
        
        # Step 3: Tokenization & POS Tagging
        pos_tags = self.pos_tagger.tag(text, lang)
        details['pos_tags'] = pos_tags
        details['steps'].append(('pos_tagging', pos_tags))
        
        # Step 4: Stopword Removal
        if remove_stopwords:
            text, removed = self.stopword_remover.remove_stopwords(
                text, lang, return_removed=True
            )
            details['removed_stopwords'] = removed
            details['steps'].append(('stopword_removal', text))
        
        # Step 5: Lemmatization
        if lemmatize:
            text = self.lemmatizer.lemmatize(text, lang)
            details['steps'].append(('lemmatization', text))
        
        # Step 6: Remove Punctuation (if requested)
        if remove_punctuation:
            # Keep only alphanumeric tokens
            tokens = text.split()
            tokens = [t for t in tokens if any(c.isalnum() for c in t)]
            text = ' '.join(tokens)
            details['steps'].append(('punctuation_removal', text))
        
        details['processed'] = text
        
        if return_details:
            return details
        return text
    
    def process_batch(self, texts, **kwargs):
        """
        Process multiple texts.
        
        Args:
            texts: List of input texts
            **kwargs: Arguments passed to process()
            
        Returns:
            list: List of processed texts
        """
        return [self.process(text, **kwargs) for text in texts]
    
    def get_pipeline_info(self):
        """Get information about the pipeline."""
        return {
            'supported_languages': self.supported_languages,
            'language_names': LANGUAGE_NAMES,
            'pipeline_stages': [
                'Language Detection',
                'Text Cleaning',
                'Tokenization',
                'POS Tagging',
                'Stopword Removal',
                'Lemmatization',
                'Punctuation Removal'
            ]
        }

# Initialize pipeline
pipeline = MultilingualNLPPipeline()

print("Pipeline Info:")
print("=" * 50)
info = pipeline.get_pipeline_info()
print(f"Supported Languages: {list(info['language_names'].values())}")
print(f"\nPipeline Stages:")
for i, stage in enumerate(info['pipeline_stages'], 1):
    print(f"  {i}. {stage}")

In [ ]:
# Test the complete pipeline
test_texts = [
    "I'm loving these amazing new features! Check https://example.com @user #NLP 😊",
    "¡Me encantan estas nuevas características increíbles! Visita https://sitio.es 🎉",
    "J'adore ces nouvelles fonctionnalités incroyables! Visitez https://site.fr 🇫🇷",
    "Ich liebe diese erstaunlichen neuen Funktionen! Besuchen Sie https://seite.de 🇩🇪"
]

print("Complete Pipeline Processing:")
print("=" * 80)

for text in test_texts:
    result = pipeline.process(text, return_details=True)
    
    print(f"\nLanguage: {LANGUAGE_NAMES.get(result['detected_language'], 'Unknown')}")
    print(f"Confidence: {result['language_confidence']:.2%}")
    print(f"\nOriginal:  {result['original']}")
    print(f"Processed: {result['processed']}")
    print(f"\nStopwords Removed: {result.get('removed_stopwords', [])[:5]}...")
    print("-" * 80)

## 9. Multilingual Embeddings

Multilingual embeddings map text from different languages into a shared vector space.

**Key Models:**
- **mBERT**: Multilingual BERT (104 languages)
- **XLM-RoBERTa**: Cross-lingual Language Model (100 languages)
- **LaBSE**: Language-agnostic BERT Sentence Embeddings

These models enable:
- Cross-lingual similarity search
- Zero-shot cross-lingual transfer
- Multilingual classification

In [ ]:
class MultilingualEmbedder:
    """
    Generate multilingual embeddings using transformer models.
    
    Models:
    - mbert: Multilingual BERT
    - xlm-roberta: XLM-RoBERTa
    - labse: Language-agnostic BERT Sentence Embeddings
    """
    
    MODEL_CONFIGS = {
        'mbert': 'bert-base-multilingual-cased',
        'xlm-roberta': 'xlm-roberta-base',
        'labse': 'sentence-transformers/LaBSE'
    }
    
    def __init__(self, model_name='xlm-roberta'):
        """
        Initialize embedder with specified model.
        
        Args:
            model_name: One of 'mbert', 'xlm-roberta', 'labse'
        """
        self.model_name = model_name
        self.model_path = self.MODEL_CONFIGS.get(model_name, model_name)
        
        print(f"Loading {model_name} model...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()
        
        print(f"Model loaded on {self.device}")
    
    def _mean_pooling(self, model_output, attention_mask):
        """Apply mean pooling to get sentence embeddings."""
        token_embeddings = model_output[0]  # First element is token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def embed(self, text):
        """
        Generate embedding for text.
        
        Args:
            text: Input text (any language)
            
        Returns:
            numpy.ndarray: Embedding vector
        """
        # Tokenize
        encoded = self.tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        encoded = {k: v.to(self.device) for k, v in encoded.items()}
        
        # Generate embeddings
        with torch.no_grad():
            output = self.model(**encoded)
        
        # Mean pooling
        embedding = self._mean_pooling(output, encoded['attention_mask'])
        
        # Normalize
        embedding = torch.nn.functional.normalize(embedding, p=2, dim=1)
        
        return embedding.cpu().numpy()[0]
    
    def embed_batch(self, texts, batch_size=32):
        """
        Generate embeddings for multiple texts.
        
        Args:
            texts: List of input texts
            batch_size: Processing batch size
            
        Returns:
            numpy.ndarray: Matrix of embeddings (n_texts x embedding_dim)
        """
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            encoded = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )
            encoded = {k: v.to(self.device) for k, v in encoded.items()}
            
            with torch.no_grad():
                output = self.model(**encoded)
            
            embeddings = self._mean_pooling(output, encoded['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
            all_embeddings.append(embeddings.cpu().numpy())
        
        return np.vstack(all_embeddings)
    
    def compute_similarity(self, text1, text2):
        """
        Compute cosine similarity between two texts.
        
        Works across languages!
        """
        emb1 = self.embed(text1)
        emb2 = self.embed(text2)
        return np.dot(emb1, emb2)

# Initialize embedder (using XLM-RoBERTa for balance of speed and quality)
print("Initializing Multilingual Embedder...")
embedder = MultilingualEmbedder('xlm-roberta')

In [ ]:
# Test cross-lingual similarity
# Same meaning in different languages should have high similarity

parallel_sentences = [
    ("I love machine learning.", "Me encanta el aprendizaje automático."),  # EN-ES
    ("The weather is beautiful today.", "Le temps est magnifique aujourd'hui."),  # EN-FR
    ("I am learning to code.", "Ich lerne programmieren."),  # EN-DE
]

unrelated_sentences = [
    ("I love machine learning.", "Le temps est magnifique aujourd'hui."),  # Different topics
    ("The cat is sleeping.", "Ich fahre mit dem Auto."),  # Cat vs Car
]

print("Cross-Lingual Similarity (Same Meaning):")
print("=" * 60)
for en, other in parallel_sentences:
    sim = embedder.compute_similarity(en, other)
    print(f"Similarity: {sim:.4f}")
    print(f"  EN: {en}")
    print(f"  --: {other}")
    print()

print("\nCross-Lingual Similarity (Different Meaning):")
print("=" * 60)
for text1, text2 in unrelated_sentences:
    sim = embedder.compute_similarity(text1, text2)
    print(f"Similarity: {sim:.4f}")
    print(f"  1: {text1}")
    print(f"  2: {text2}")
    print()

In [ ]:
# Visualize embeddings in 2D
from sklearn.decomposition import PCA

# Sentences about similar topics in different languages
sentences = [
    # Technology (cluster 1)
    "Artificial intelligence is transforming the world.",
    "La inteligencia artificial está transformando el mundo.",
    "L'intelligence artificielle transforme le monde.",
    "Künstliche Intelligenz verändert die Welt.",
    
    # Nature (cluster 2)
    "The forest is full of beautiful trees.",
    "El bosque está lleno de árboles hermosos.",
    "La forêt est pleine de beaux arbres.",
    "Der Wald ist voller schöner Bäume.",
    
    # Food (cluster 3)
    "I enjoy eating pizza and pasta.",
    "Me gusta comer pizza y pasta.",
    "J'aime manger de la pizza et des pâtes.",
    "Ich esse gerne Pizza und Pasta."
]

labels = ['EN-Tech', 'ES-Tech', 'FR-Tech', 'DE-Tech',
          'EN-Nature', 'ES-Nature', 'FR-Nature', 'DE-Nature',
          'EN-Food', 'ES-Food', 'FR-Food', 'DE-Food']

topics = ['Technology'] * 4 + ['Nature'] * 4 + ['Food'] * 4
languages = ['English', 'Spanish', 'French', 'German'] * 3

# Generate embeddings
embeddings = embedder.embed_batch(sentences)

# Reduce to 2D
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

colors = {'Technology': 'red', 'Nature': 'green', 'Food': 'blue'}
markers = {'English': 'o', 'Spanish': 's', 'French': '^', 'German': 'D'}

for i, (x, y) in enumerate(embeddings_2d):
    ax.scatter(x, y, 
               c=colors[topics[i]], 
               marker=markers[languages[i]],
               s=150,
               alpha=0.7)
    ax.annotate(labels[i], (x, y), fontsize=9, 
                xytext=(5, 5), textcoords='offset points')

# Legend
from matplotlib.lines import Line2D
topic_handles = [Line2D([0], [0], marker='o', color='w', 
                        markerfacecolor=c, markersize=10, label=t)
                 for t, c in colors.items()]
lang_handles = [Line2D([0], [0], marker=m, color='gray', 
                       markersize=10, label=l, linestyle='None')
                for l, m in markers.items()]

ax.legend(handles=topic_handles + lang_handles, 
          loc='upper left', bbox_to_anchor=(1.02, 1))

ax.set_title('Multilingual Embeddings: Topics Cluster Across Languages')
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
plt.tight_layout()
plt.show()

print("\nObservation: Sentences about the same topic cluster together,")
print("regardless of language! This is the power of multilingual embeddings.")

## 10. Cross-Lingual Text Classification

One powerful application of multilingual embeddings is **zero-shot cross-lingual transfer**:
- Train a classifier in ONE language (e.g., English)
- Apply it to OTHER languages without any training data in those languages!

This is incredibly useful when you have labeled data in only one language.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

class CrossLingualClassifier:
    """
    Train on one language, predict on any language.
    
    Uses multilingual embeddings for zero-shot cross-lingual transfer.
    """
    
    def __init__(self, embedder):
        self.embedder = embedder
        self.classifier = LogisticRegression(max_iter=1000)
        self.is_trained = False
        self.classes = None
    
    def train(self, texts, labels):
        """
        Train classifier on labeled texts.
        
        Args:
            texts: List of training texts
            labels: List of labels
        """
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.embedder.embed_batch(texts)
        
        print("Training classifier...")
        self.classifier.fit(embeddings, labels)
        self.classes = self.classifier.classes_
        self.is_trained = True
        print("Training complete!")
    
    def predict(self, texts):
        """
        Predict labels for texts (any language!).
        
        Args:
            texts: List of texts to classify
            
        Returns:
            numpy.ndarray: Predicted labels
        """
        if not self.is_trained:
            raise ValueError("Classifier not trained. Call train() first.")
        
        embeddings = self.embedder.embed_batch(texts)
        return self.classifier.predict(embeddings)
    
    def predict_proba(self, texts):
        """Get prediction probabilities."""
        if not self.is_trained:
            raise ValueError("Classifier not trained. Call train() first.")
        
        embeddings = self.embedder.embed_batch(texts)
        return self.classifier.predict_proba(embeddings)
    
    def evaluate(self, texts, labels):
        """
        Evaluate classifier on test data.
        
        Returns:
            dict: Evaluation metrics
        """
        predictions = self.predict(texts)
        accuracy = accuracy_score(labels, predictions)
        
        return {
            'accuracy': accuracy,
            'predictions': predictions,
            'report': classification_report(labels, predictions)
        }

# Create training data (English only)
train_texts_en = [
    # Positive sentiment
    "I love this product, it's amazing!",
    "This is the best thing I've ever bought.",
    "Absolutely fantastic experience!",
    "Great quality and fast delivery.",
    "I'm so happy with this purchase.",
    "Excellent service, highly recommend!",
    "This exceeded all my expectations.",
    "Perfect, exactly what I needed.",
    
    # Negative sentiment
    "Terrible product, waste of money.",
    "I'm very disappointed with this.",
    "The worst purchase I've ever made.",
    "Poor quality, broke after one day.",
    "Don't buy this, it's awful.",
    "Horrible experience, never again.",
    "Complete garbage, avoid at all costs.",
    "This product is defective and useless."
]

train_labels = ['positive'] * 8 + ['negative'] * 8

# Initialize and train
classifier = CrossLingualClassifier(embedder)
classifier.train(train_texts_en, train_labels)

In [ ]:
# Test on multiple languages (zero-shot transfer!)
test_data = {
    'English': [
        ("This is wonderful, I love it!", 'positive'),
        ("Terrible quality, very bad.", 'negative'),
    ],
    'Spanish': [
        ("¡Esto es maravilloso, me encanta!", 'positive'),
        ("Calidad terrible, muy malo.", 'negative'),
    ],
    'French': [
        ("C'est merveilleux, j'adore!", 'positive'),
        ("Qualité terrible, très mauvais.", 'negative'),
    ],
    'German': [
        ("Das ist wunderbar, ich liebe es!", 'positive'),
        ("Schreckliche Qualität, sehr schlecht.", 'negative'),
    ],
    'Italian': [
        ("Questo è meraviglioso, lo adoro!", 'positive'),
        ("Qualità terribile, molto cattivo.", 'negative'),
    ]
}

print("Zero-Shot Cross-Lingual Classification Results:")
print("=" * 60)
print("(Trained ONLY on English data)\n")

results = {}
for lang, examples in test_data.items():
    texts = [t for t, _ in examples]
    true_labels = [l for _, l in examples]
    
    predictions = classifier.predict(texts)
    accuracy = accuracy_score(true_labels, predictions)
    results[lang] = accuracy
    
    print(f"{lang}:")
    for text, true, pred in zip(texts, true_labels, predictions):
        status = "✓" if true == pred else "✗"
        print(f"  {status} '{text[:40]}...'")
        print(f"    True: {true}, Pred: {pred}")
    print(f"  Accuracy: {accuracy:.0%}\n")

# Summary
print("\nSummary - Accuracy by Language:")
print("-" * 30)
for lang, acc in results.items():
    bar = "█" * int(acc * 20)
    print(f"{lang:10} {bar} {acc:.0%}")

## 11. Translation System

Machine translation is another key component of multilingual NLP. We'll use MarianMT models from Helsinki-NLP, which are compact and effective.

**Model naming convention:** `Helsinki-NLP/opus-mt-{src}-{tgt}`
- `en-es`: English to Spanish
- `es-en`: Spanish to English
- etc.

In [ ]:
class MultilingualTranslator:
    """
    Translate between multiple languages using MarianMT.
    
    Supports: English, Spanish, French, German, Italian, Portuguese
    """
    
    def __init__(self):
        self.models = {}
        self.tokenizers = {}
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Available translation pairs
        self.supported_pairs = [
            ('en', 'es'), ('es', 'en'),
            ('en', 'fr'), ('fr', 'en'),
            ('en', 'de'), ('de', 'en'),
            ('en', 'it'), ('it', 'en'),
            ('en', 'pt'), ('pt', 'en'),
        ]
    
    def _get_model_name(self, src, tgt):
        """Get HuggingFace model name for language pair."""
        return f"Helsinki-NLP/opus-mt-{src}-{tgt}"
    
    def _load_model(self, src, tgt):
        """Load translation model for language pair."""
        pair = (src, tgt)
        if pair not in self.models:
            model_name = self._get_model_name(src, tgt)
            print(f"Loading translation model: {src} → {tgt}...")
            
            self.tokenizers[pair] = MarianTokenizer.from_pretrained(model_name)
            self.models[pair] = MarianMTModel.from_pretrained(model_name)
            self.models[pair].to(self.device)
            self.models[pair].eval()
        
        return self.models[pair], self.tokenizers[pair]
    
    def translate(self, text, src_lang, tgt_lang):
        """
        Translate text from source to target language.
        
        Args:
            text: Input text
            src_lang: Source language code
            tgt_lang: Target language code
            
        Returns:
            str: Translated text
        """
        if (src_lang, tgt_lang) not in self.supported_pairs:
            raise ValueError(f"Translation pair {src_lang} → {tgt_lang} not supported")
        
        model, tokenizer = self._load_model(src_lang, tgt_lang)
        
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Generate translation
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=512)
        
        # Decode
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translation
    
    def translate_batch(self, texts, src_lang, tgt_lang):
        """Translate multiple texts."""
        return [self.translate(text, src_lang, tgt_lang) for text in texts]
    
    def translate_to_english(self, text, src_lang):
        """Convenience method to translate any language to English."""
        if src_lang == 'en':
            return text
        return self.translate(text, src_lang, 'en')
    
    def get_supported_pairs(self):
        """Get list of supported translation pairs."""
        return self.supported_pairs

# Initialize translator
translator = MultilingualTranslator()

print("Supported Translation Pairs:")
for src, tgt in translator.get_supported_pairs():
    print(f"  {LANGUAGE_NAMES[src]} → {LANGUAGE_NAMES[tgt]}")

In [ ]:
# Test translations
test_text = "Machine learning is revolutionizing how we solve complex problems."

print(f"Original (English): {test_text}")
print("=" * 70)

target_languages = ['es', 'fr', 'de', 'it', 'pt']

translations = {}
for tgt in target_languages:
    translation = translator.translate(test_text, 'en', tgt)
    translations[tgt] = translation
    print(f"{LANGUAGE_NAMES[tgt]:12}: {translation}")

In [ ]:
# Round-trip translation test (translate and back)
print("Round-Trip Translation Test:")
print("(English → Target → English)")
print("=" * 70)

original = "The quick brown fox jumps over the lazy dog."
print(f"\nOriginal: {original}\n")

for lang in ['es', 'fr', 'de']:
    # Translate to target language
    translated = translator.translate(original, 'en', lang)
    # Translate back to English
    back_translated = translator.translate(translated, lang, 'en')
    
    print(f"{LANGUAGE_NAMES[lang]}:")
    print(f"  → {translated}")
    print(f"  ← {back_translated}")
    print()

## 12. Summary

We've built a complete **Multilingual NLP Pipeline** covering:

### Core Pipeline Components

| Component | Purpose | Key Library |
|-----------|---------|-------------|
| Language Detection | Identify text language | langdetect |
| Tokenization | Split text into tokens | spaCy |
| Text Cleaning | Remove noise, normalize | regex, unicodedata |
| POS Tagging | Identify parts of speech | spaCy |
| Stopword Removal | Remove common words | NLTK + spaCy |
| Lemmatization | Reduce to base form | spaCy |

### Advanced Features

| Feature | Purpose | Model |
|---------|---------|-------|
| Multilingual Embeddings | Shared vector space | XLM-RoBERTa |
| Cross-lingual Classification | Zero-shot transfer | Embeddings + LogReg |
| Machine Translation | Text translation | MarianMT |

### Key Takeaways

1. **Language-specific resources matter**: Different languages need different tokenization, stopwords, and lemmatization rules

2. **Universal tagsets help**: POS tags using Universal Dependencies work across languages

3. **Multilingual embeddings are powerful**: They map semantically similar content close together, regardless of language

4. **Zero-shot transfer works**: Train in one language, apply to many others

5. **Translation enables more**: When you need language-specific processing, translate first

In [ ]:
# Final demonstration: Complete workflow
print("="*70)
print("COMPLETE MULTILINGUAL NLP WORKFLOW DEMONSTRATION")
print("="*70)

# Input: Mixed language texts
input_texts = [
    "I absolutely love this new AI technology! It's revolutionary.",
    "¡Esta tecnología de IA es increíble! Cambiará todo.",
    "Cette technologie d'IA est incroyable! Elle va tout changer.",
    "Diese KI-Technologie ist unglaublich! Sie wird alles verändern."
]

print("\n1. INPUT TEXTS (Multiple Languages):")
print("-" * 50)
for i, text in enumerate(input_texts, 1):
    print(f"   {i}. {text}")

print("\n2. LANGUAGE DETECTION:")
print("-" * 50)
detected_langs = []
for text in input_texts:
    lang, conf = lang_detector.detect(text)
    detected_langs.append(lang)
    print(f"   {LANGUAGE_NAMES.get(lang, lang):12} ({conf:.0%}): {text[:40]}...")

print("\n3. PREPROCESSING (Pipeline):")
print("-" * 50)
processed_texts = []
for text, lang in zip(input_texts, detected_langs):
    processed = pipeline.process(text, lang=lang)
    processed_texts.append(processed)
    print(f"   {LANGUAGE_NAMES.get(lang, lang):12}: {processed}")

print("\n4. EMBEDDINGS & SIMILARITY:")
print("-" * 50)
embeddings = embedder.embed_batch(input_texts)
print(f"   Generated {embeddings.shape[0]} embeddings of dimension {embeddings.shape[1]}")

# Compute pairwise similarity
print("   \n   Cross-lingual similarity matrix:")
langs_short = ['EN', 'ES', 'FR', 'DE']
print(f"        {' '.join([f'{l:6}' for l in langs_short])}")
for i, lang in enumerate(langs_short):
    sims = [np.dot(embeddings[i], embeddings[j]) for j in range(len(embeddings))]
    sim_str = ' '.join([f'{s:.3f} ' for s in sims])
    print(f"   {lang}   {sim_str}")

print("\n5. SENTIMENT CLASSIFICATION (Zero-shot):")
print("-" * 50)
predictions = classifier.predict(input_texts)
for text, lang, pred in zip(input_texts, detected_langs, predictions):
    print(f"   {LANGUAGE_NAMES.get(lang, lang):12}: {pred:8} | {text[:35]}...")

print("\n" + "="*70)
print("Pipeline complete! All texts processed through the multilingual NLP system.")
print("="*70)

In [ ]:
# Save pipeline components for reuse
print("\nPipeline Components Summary:")
print("="*50)
print("""
Classes implemented:
--------------------
1. LanguageDetector       - Detect language of text
2. MultilingualTokenizer  - Language-aware tokenization
3. MultilingualTextCleaner - Clean text preserving unicode
4. MultilingualPOSTagger  - Universal POS tagging
5. MultilingualStopwordRemover - Language-specific stopwords
6. MultilingualLemmatizer - Reduce words to base forms
7. MultilingualNLPPipeline - Complete preprocessing pipeline
8. MultilingualEmbedder   - Generate cross-lingual embeddings
9. CrossLingualClassifier - Train once, classify any language
10. MultilingualTranslator - Translate between languages

Supported Languages:
-------------------
- English (en)
- Spanish (es)
- French (fr)
- German (de)
- Italian (it)
- Portuguese (pt)

Key Models Used:
---------------
- spaCy multilingual models (for NLP processing)
- XLM-RoBERTa (for embeddings)
- MarianMT (for translation)
""")